In [19]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os


# Using energy Source Datasets

# Source of Energy Petroleum

In [20]:
# Read the Excel file
# Data_Status: Indicates the status of the data. The value "2020F" suggests that it is a forecast for the year 2020.
# State: Represents the state for which the data is recorded (in this case, "CA" for California).
# MSN: Stands for "Monthly State Names" and refers to the specific energy metric or variable being measured. Examples include ARICD, ARICV, ARTCD, ARTCV, ARTXD, WWTXV, WXICD, WXICV, ZWCDP, ZWHDP.
df = pd.read_excel('Datasets/use_energy_source.xlsx' , sheet_name='Nuclear')


In [21]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AL,0,0,0,0,0,0,0,0,0,...,411822,427975,426488,431368,438728,417334,446097,412584,455865,454756
2,AR,0,0,0,0,0,0,0,0,0,...,148531,162353,124816,151428,144716,140370,132736,132996,141750,157288
3,AZ,0,0,0,0,0,0,0,0,0,...,327292,334636,328423,338044,340153,338636,338246,325122,333312,329471
4,CA,1,55,84,2288,4369,3193,1895,6503,17003,...,383644,193939,187162,177656,193529,197754,187227,190422,168799,169773


# Data statistics

In [22]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AL,0,0,0,0,0,0,0,0,0,...,411822,427975,426488,431368,438728,417334,446097,412584,455865,454756
2,AR,0,0,0,0,0,0,0,0,0,...,148531,162353,124816,151428,144716,140370,132736,132996,141750,157288
3,AZ,0,0,0,0,0,0,0,0,0,...,327292,334636,328423,338044,340153,338636,338246,325122,333312,329471
4,CA,1,55,84,2288,4369,3193,1895,6503,17003,...,383644,193939,187162,177656,193529,197754,187227,190422,168799,169773


In [23]:
df.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,...,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01
mean,231.788462,756.846154,1015.153846,1467.192308,1531.519231,1660.173077,2467.615385,3402.153846,5443.615385,5912.384615,...,3.180269e+05,3.100701e+05,3.170936e+05,3.206753e+05,3.206495e+05,3.241058e+05,3.238065e+05,3.245411e+05,3.250712e+05,3.172275e+05
std,985.922547,3151.103363,4233.447732,5716.353205,6020.306407,6394.706851,9396.676348,12841.396647,20944.051365,22700.690298,...,1.144491e+06,1.116091e+06,1.141532e+06,1.153955e+06,1.154280e+06,1.167086e+06,1.165964e+06,1.168542e+06,1.170826e+06,1.142808e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,-64.000000,0.000000,0.000000,-30.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.204650e+04,8.631600e+04,8.791800e+04,9.906950e+04,1.019260e+05,9.821400e+04,8.593300e+04,9.867800e+04,9.626300e+04,8.964000e+04
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.930845e+05,2.706885e+05,2.988475e+05,2.976258e+05,2.936160e+05,3.097592e+05,3.228662e+05,3.095095e+05,3.049628e+05,3.090782e+05
max,6026.000000,19678.000000,26394.000000,38147.000000,39819.000000,43164.000000,64158.000000,88456.000000,141534.000000,153722.000000,...,8.268698e+06,8.061822e+06,8.244433e+06,8.337559e+06,8.336886e+06,8.426753e+06,8.418968e+06,8.438068e+06,8.451852e+06,8.247915e+06


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 62 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   52 non-null     object
 1   1960    52 non-null     int64 
 2   1961    52 non-null     int64 
 3   1962    52 non-null     int64 
 4   1963    52 non-null     int64 
 5   1964    52 non-null     int64 
 6   1965    52 non-null     int64 
 7   1966    52 non-null     int64 
 8   1967    52 non-null     int64 
 9   1968    52 non-null     int64 
 10  1969    52 non-null     int64 
 11  1970    52 non-null     int64 
 12  1971    52 non-null     int64 
 13  1972    52 non-null     int64 
 14  1973    52 non-null     int64 
 15  1974    52 non-null     int64 
 16  1975    52 non-null     int64 
 17  1976    52 non-null     int64 
 18  1977    52 non-null     int64 
 19  1978    52 non-null     int64 
 20  1979    52 non-null     int64 
 21  1980    52 non-null     int64 
 22  1981    52 non-null     int6

In [25]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AL,0,0,0,0,0,0,0,0,0,...,411822,427975,426488,431368,438728,417334,446097,412584,455865,454756
2,AR,0,0,0,0,0,0,0,0,0,...,148531,162353,124816,151428,144716,140370,132736,132996,141750,157288
3,AZ,0,0,0,0,0,0,0,0,0,...,327292,334636,328423,338044,340153,338636,338246,325122,333312,329471
4,CA,1,55,84,2288,4369,3193,1895,6503,17003,...,383644,193939,187162,177656,193529,197754,187227,190422,168799,169773


# Transformation of Data

In [26]:
df_trans = df.melt(id_vars=['State'] , var_name='Year', value_name='Yearly Data')
df_trans['Year'] = pd.to_datetime(df_trans['Year'], format='%Y')

df_trans


,State,Year,Yearly Data
0,AK,1960-01-01,0
1,AL,1960-01-01,0
2,AR,1960-01-01,0
3,AZ,1960-01-01,0
4,CA,1960-01-01,1
...,...,...,...
3167,WA,2020-01-01,98437
3168,WI,2020-01-01,102032
3169,WV,2020-01-01,0
3170,WY,2020-01-01,0


# Modeling the Data
## Using ARIMA Model

In [27]:
os.makedirs('Plots/use_energy_source_Nuclear/Arima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Nuclear : Energy Consumption Forecast State : {State} using ARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_energy_source_Nuclear/Arima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        

/home/iffi/anaconda3/envs/sep_darts_2/lib/python3.11/site-packages/pmdarima/arima/auto.py:444: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

/tmp/ipykernel_37611/998026100.py:36: RuntimeWarning:

invalid value encountered in divide



Mean Absolute Error (MAE): 0.0
Mean Squared Error (MSE): 0.0
Mean Absolute Percentage Error (MAPE): nan


Mean Absolute Error (MAE): 25331.42
Mean Squared Error (MSE): 846221378.6
Mean Absolute Percentage Error (MAPE): 5.92


Mean Absolute Error (MAE): 14986.28
Mean Squared Error (MSE): 272709820.9
Mean Absolute Percentage Error (MAPE): 10.94


Mean Absolute Error (MAE): 27181.65
Mean Squared Error (MSE): 901330935.89
Mean Absolute Percentage Error (MAPE): 8.22


...

Mean Absolute Error (MAE): 0.0
Mean Squared Error (MSE): 0.0
Mean Absolute Percentage Error (MAPE): nan


/home/iffi/anaconda3/envs/sep_darts_2/lib/python3.11/site-packages/pmdarima/arima/auto.py:444: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

/tmp/ipykernel_37611/998026100.py:36: RuntimeWarning:

invalid value encountered in divide



Mean Absolute Error (MAE): 0.0
Mean Squared Error (MSE): 0.0
Mean Absolute Percentage Error (MAPE): nan


/home/iffi/anaconda3/envs/sep_darts_2/lib/python3.11/site-packages/pmdarima/arima/auto.py:444: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

/tmp/ipykernel_37611/998026100.py:36: RuntimeWarning:

invalid value encountered in divide



Mean Absolute Error (MAE): 341992.24
Mean Squared Error (MSE): 183990705394.7
Mean Absolute Percentage Error (MAPE): 4.1


## Using SARIMA Model

In [28]:
os.makedirs('Plots/use_energy_source_Nuclear/Sarima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Nuclear : Energy Consumption Forecast State : {State} using SARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_energy_source_Nuclear/Sarima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        # break

/home/iffi/anaconda3/envs/sep_darts_2/lib/python3.11/site-packages/pmdarima/arima/auto.py:444: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

/tmp/ipykernel_37611/815027103.py:36: RuntimeWarning:

invalid value encountered in divide



Mean Absolute Error (MAE): 0.0
Mean Squared Error (MSE): 0.0
Mean Absolute Percentage Error (MAPE): nan


Mean Absolute Error (MAE): 25331.42
Mean Squared Error (MSE): 846221378.6
Mean Absolute Percentage Error (MAPE): 5.92


Mean Absolute Error (MAE): 14986.28
Mean Squared Error (MSE): 272709820.9
Mean Absolute Percentage Error (MAPE): 10.94


Mean Absolute Error (MAE): 27181.65
Mean Squared Error (MSE): 901330935.89
Mean Absolute Percentage Error (MAPE): 8.22


...

Mean Absolute Error (MAE): 0.0
Mean Squared Error (MSE): 0.0
Mean Absolute Percentage Error (MAPE): nan


/home/iffi/anaconda3/envs/sep_darts_2/lib/python3.11/site-packages/pmdarima/arima/auto.py:444: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

/tmp/ipykernel_37611/815027103.py:36: RuntimeWarning:

invalid value encountered in divide



Mean Absolute Error (MAE): 0.0
Mean Squared Error (MSE): 0.0
Mean Absolute Percentage Error (MAPE): nan


/home/iffi/anaconda3/envs/sep_darts_2/lib/python3.11/site-packages/pmdarima/arima/auto.py:444: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

/tmp/ipykernel_37611/815027103.py:36: RuntimeWarning:

invalid value encountered in divide



Mean Absolute Error (MAE): 341992.24
Mean Squared Error (MSE): 183990705394.7
Mean Absolute Percentage Error (MAPE): 4.1
